Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import emoji
from nltk import word_tokenize
from string import punctuation, ascii_letters
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from gensim.models import Word2Vec

In [2]:
MAX_WORDS = 2000
MAX_LEN = 20
EMB_SIZE = 100

In [3]:
morph = MorphAnalyzer(lang='ru')

In [4]:
stop_words = set(get_stop_words('ru'))

In [5]:
punctuation = set(punctuation).union((' ', '«', '»', '—', '–', '“', '”', '…'))

In [6]:
cyrillic_letters = set([chr(i) for i in range(ord('а'), ord('я') + 1)] +
                       [chr(i) for i in range(ord('А'), ord('Я') + 1)] +
                       ['ё', 'Ё'])

In [7]:
data = pd.read_excel('../data/summer_reviews.xls')
data.columns = ['rating', 'content', 'date']

In [8]:
data['target'] = (data.rating > 3).astype(int)

In [9]:
def preprocess(text):
    tokens = word_tokenize(text)
    result = []
    for token in tokens:
        if (set(token).intersection(cyrillic_letters)
            or set(token).intersection(set(ascii_letters))
            or token in emoji.UNICODE_EMOJI):
            result.append(token)
    tokens = [token.lower() for token in result if token.lower() not in stop_words]
    tokens = [token for token in tokens if token in emoji.UNICODE_EMOJI or len(token) >  1]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens
        

In [10]:
data['processed'] = data.content.apply(lambda x: " ".join(preprocess(str(x))))

In [11]:
X_train, X_val, y_train, y_val = train_test_split(data['processed'], data['target'], test_size=0.1,
                                                    random_state=0, stratify=data['target'])

In [12]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3000, oov_token='UNK')

In [13]:
tokenizer.fit_on_texts(X_train)

In [14]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [15]:
X_val = pad_sequences(tokenizer.texts_to_sequences(X_val),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [16]:
vocab = tokenizer.index_word

### RNN

In [17]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
rnn = tf.keras.layers.SimpleRNN(128, recurrent_dropout=0.2, activation='relu')(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(rnn)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [18]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [20]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 3s 170us/sample - loss: 0.4631 - accuracy: 0.8076 - val_loss: 0.3634 - val_accuracy: 0.8621
Epoch 2/10
18593/18593 [==============================] - 2s 120us/sample - loss: 0.2929 - accuracy: 0.8789 - val_loss: 0.3645 - val_accuracy: 0.9032
Epoch 3/10
18593/18593 [==============================] - 2s 116us/sample - loss: 0.2494 - accuracy: 0.9010 - val_loss: 0.3016 - val_accuracy: 0.8925
Epoch 4/10
18593/18593 [==============================] - 2s 120us/sample - loss: 0.2881 - accuracy: 0.8412 - val_loss: 0.3748 - val_accuracy: 0.8170
Epoch 5/10
18593/18593 [==============================] - 2s 118us/sample - loss: 0.2553 - accuracy: 0.8931 - val_loss: 0.3008 - val_accuracy: 0.8824
Epoch 6/10
18593/18593 [==============================] - 2s 112us/sample - loss: 0.2231 - accuracy: 0.8996 - val_loss: 0.2924 - val_accuracy: 0.8911
Epoch 7/10
18593/18593 [===========================

### Bidirectional LSTM

In [22]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(lstm)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [23]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [25]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 5s 287us/sample - loss: 0.3606 - accuracy: 0.8462 - val_loss: 0.2057 - val_accuracy: 0.9066
Epoch 2/10
18593/18593 [==============================] - 3s 179us/sample - loss: 0.1995 - accuracy: 0.9192 - val_loss: 0.1848 - val_accuracy: 0.9109
Epoch 3/10
18593/18593 [==============================] - 3s 182us/sample - loss: 0.1744 - accuracy: 0.9312 - val_loss: 0.1884 - val_accuracy: 0.9163
Epoch 4/10
18593/18593 [==============================] - 4s 189us/sample - loss: 0.1602 - accuracy: 0.9380 - val_loss: 0.1990 - val_accuracy: 0.9100
Epoch 5/10
18593/18593 [==============================] - 4s 197us/sample - loss: 0.1461 - accuracy: 0.9443 - val_loss: 0.1994 - val_accuracy: 0.9134
Epoch 6/10
18593/18593 [==============================] - 3s 179us/sample - loss: 0.1316 - accuracy: 0.9493 - val_loss: 0.2213 - val_accuracy: 0.9085
Epoch 7/10
18593/18593 [===========================

### GRU

In [27]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(gru)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [28]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [30]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 18593 samples, validate on 2066 samples
Epoch 1/10
18593/18593 [==============================] - 5s 258us/sample - loss: 0.3524 - accuracy: 0.8478 - val_loss: 0.2031 - val_accuracy: 0.9066
Epoch 2/10
18593/18593 [==============================] - 3s 159us/sample - loss: 0.1983 - accuracy: 0.9177 - val_loss: 0.1863 - val_accuracy: 0.9129
Epoch 3/10
18593/18593 [==============================] - 3s 160us/sample - loss: 0.1740 - accuracy: 0.9316 - val_loss: 0.1876 - val_accuracy: 0.9124
Epoch 4/10
18593/18593 [==============================] - 3s 160us/sample - loss: 0.1606 - accuracy: 0.9377 - val_loss: 0.1948 - val_accuracy: 0.9134
Epoch 5/10
18593/18593 [==============================] - 3s 156us/sample - loss: 0.1504 - accuracy: 0.9429 - val_loss: 0.2061 - val_accuracy: 0.9095
Epoch 6/10
18593/18593 [==============================] - 3s 162us/sample - loss: 0.1396 - accuracy: 0.9483 - val_loss: 0.2067 - val_accuracy: 0.9105
Epoch 7/10
18593/18593 [===========================

Как видим, на валидации у всех моделей результат схожий. Если смотреть на лучшую эпоху, то небольшое преимущество у LSTM с результатом `0.9163`.